---
title: "Die Räume"
jupyter: python3
#code-fold: true
execute:
  echo: false
  output: asis
---

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pprint 
from pathlib import Path
from collections import defaultdict

def query_WB(endpoint, query):

    # Initialize the wrapper
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)

    # Fetch and parse results
    results = sparql.query().convert()

    return results 

def get_data(results):

    grouped = defaultdict(list)

    # Display results
    for result in results["results"]["bindings"]:
        url = result["item"]["value"]
        label = result.get("itemLabel", {}).get("value", "")
        photo = result.get("photo", {}).get("value", "")
        descr = result.get("itemDescription", {}).get("value", "")
        room_url = result["room"]["value"]
        room = result["roomLabel"]["value"]
        photographer = result.get("creator", {}).get("value", "") 

        key = (room, room_url)

        grouped[key].append({
          "photo": photo,
          "photographer": photographer,
          "url": url,
          "label": label,
          "descr": descr
        })

    return grouped

def is_supported_image(path):
    ext = Path(path).suffix.lower()
    return ext in [".jpg", ".jpeg", ".png"]

def generate_output(grouped):

    for (room, room_url), entries in grouped.items():
        print(f"""
## {room}\\index{{{room}}}

Wikibase-Eintrag: [{room} (Raum)]({room_url})

""")
        for entry in entries:
            print(f"""
### {entry["label"]}\\index{{{room}!{entry["label"]}}}
            """)

            if not is_supported_image(entry["photo"]):
                print(f"""
                      
*Bildformat nicht unterstützt!*

            """) 

            else:
                print(f"""
                      
![Photo: {entry['photographer']}]({entry['photo']}){{group="room-gallery"}}

Bildquelle: [{entry["photo"]}]({entry["photo"]})

                """)
            print(f"""
Wikibase-Eintrag: [{entry["label"]} (Kunstwerk)]({entry["url"]})  
            """) 

        print(f"\\newpage")




In [2]:
def make_book():

    endpoint_url = "https://query.kewl.org/sparql"

    query = """
        PREFIX wd: <https://wikibase.kewl.org/entity/>
        PREFIX wdt: <https://wikibase.kewl.org/prop/direct/>
        PREFIX p: <https://wikibase.kewl.org/prop/>
        PREFIX ps: <https://wikibase.kewl.org/prop/statement/>
        PREFIX pq: <https://wikibase.kewl.org/prop/qualifier/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX bd: <http://www.bigdata.com/rdf#>

        SELECT ?item ?itemLabel ?photo ?itemDescription ?room ?roomLabel ?creator WHERE {
            VALUES ?castles { wd:Q68 wd:Q10 }
            ?item wdt:P3+ ?castles ;
                    wdt:P3 ?room ;
                    wdt:P1 wd:Q6 .
            ?item p:P6 ?statement .
            ?statement ps:P6 ?photo .
            OPTIONAL { ?statement pq:P11 ?creator. }

            SERVICE wikibase:label {
                bd:serviceParam wikibase:language "de"
            }
        } ORDER BY ?roomLabel
    """


    results = query_WB(endpoint_url,query)
    grouped = get_data(results)
    generate_output(grouped)

make_book()


## Das Vorzimmer der Äbtissin\index{Das Vorzimmer der Äbtissin}

Wikibase-Eintrag: [Das Vorzimmer der Äbtissin (Raum)](https://wikibase.kewl.org/entity/Q11)



### Die Deckenmalereireste\index{Das Vorzimmer der Äbtissin!Die Deckenmalereireste}
            

                      
![Photo: Bunz, Achim](https://previous.bildindex.de/bilder/fmd10045421a.jpg){group="room-gallery"}

Bildquelle: [https://previous.bildindex.de/bilder/fmd10045421a.jpg](https://previous.bildindex.de/bilder/fmd10045421a.jpg)

                

Wikibase-Eintrag: [Die Deckenmalereireste (Kunstwerk)](https://wikibase.kewl.org/entity/Q12)  
            
\newpage

## Der Winterremter\index{Der Winterremter}

Wikibase-Eintrag: [Der Winterremter (Raum)](https://wikibase.kewl.org/entity/Q64)



### Die ältere Wandmalerei\index{Der Winterremter!Die ältere Wandmalerei}
            

                      
![Photo: Bunz, Achim](https://previous.bildindex.de/bilder/fmd10045436a.jpg){group="room-gallery"}

Bildquelle: [http